<a href="https://colab.research.google.com/github/santhanreddy864/In-class-exercise-1/blob/main/In_class_exercise_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The fifth In-class-exercise (2/23/2021, 20 points in total)

In exercise-03, I asked you to collected 500 textual data based on your own information needs (If you didn't collect the textual data, you should recollect for this exercise). Now we need to think about how to represent the textual data for text classification. In this exercise, you are required to select 10 types of features (10 types of features but absolutely more than 10 features) in the followings feature list, then represent the 500 texts with these features. The output should be in the following format:
![image.png](attachment:image.png)

The feature list:

* (1) tf-idf features
* (2) POS-tag features: number of adjective, adverb, auxiliary, punctuation, complementizer, coordinating conjunction, subordinating conjunction, determiner, interjection, noun, possessor, preposition, pronoun, quantifier, verb, and other. (select some of them if you use pos-tag features)
* (3) Linguistic features:
  * number of right-branching nodes across all constituent types
  * number of right-branching nodes for NPs only
  * number of left-branching nodes across all constituent types
  * number of left-branching nodes for NPs only
  * number of premodifiers across all constituent types
  * number of premodifiers within NPs only
  * number of postmodifiers across all constituent types
  * number of postmodifiers within NPs only
  * branching index across all constituent types, i.e. the number of right-branching nodes minus number of left-branching nodes
  * branching index for NPs only
  * branching weight index: number of tokens covered by right-branching nodes minus number of tokens covered by left-branching nodes across all categories
  * branching weight index for NPs only 
  * modification index, i.e. the number of premodifiers minus the number of postmodifiers across all categories
  * modification index for NPs only
  * modification weight index: length in tokens of all premodifiers minus length in tokens of all postmodifiers across all categories
  * modification weight index for NPs only
  * coordination balance, i.e. the maximal length difference in coordinated constituents
  
  * density (density can be calculated using the ratio of folowing function words to content words) of determiners/quantifiers
  * density of pronouns
  * density of prepositions
  * density of punctuation marks, specifically commas and semicolons
  * density of auxiliary verbs
  * density of conjunctions
  * density of different pronoun types: Wh, 1st, 2nd, and 3rd person pronouns
  
  * maximal and average NP length
  * maximal and average AJP length
  * maximal and average PP length
  * maximal and average AVP length
  * sentence length

* Other features in your mind (ie., pre-defined patterns)

In [11]:
#Data from the inclass exercise3
import requests 
import json 
import time
from time import sleep
import random
import pandas as pd
from bs4 import BeautifulSoup
l1 = []
l2 = []
for no in range(52):
  link = "https://www.amazon.com/Apple-iPad-Air-10-9-inch-Wi-Fi-64GB/product-reviews/B08J61FCVN/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews" + str(no)
  page = requests.get(link)
  soup = BeautifulSoup(page.text, 'html.parser')
  head_review = soup.find_all(class_='a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold')
  description_review = soup.find_all(class_='a-size-base review-text review-text-content')
  for element, l2_element in zip(head_review, description_review) :
    l1.append(element.text)
    l2.append(l2_element.text)
df = pd.DataFrame(list(zip(l1, l2)), columns =['overview of customer review', 'detailed Review'])
print("Choosen data frame length is: {0}".format(len(df))) 
df

Choosen data frame length is: 10


,overview of customer review,detailed Review
0,\nSmall improvements combine for a great exper...,"\n\n This new iPad Air, on the surface, isn't..."
1,"\nGreat product, delivery concerning\n","\n\n Having had ipad , ipad air 2, I would ha..."
2,\nnot to user friendly for person over 65 olds...,"\n\n doesn't work that great slow, to many ap..."
3,"\nGreat for the casuals, and then some\n",\n\n I purchased this as a sort-of alternativ...
4,\niPad Air 4 Rocks\n,\n\n I held off replacing my iPad Air 2 in ho...
5,\nThe New iPad Air Is A Hit!!\n,\n\n I think this particular iPad will be a p...
6,\nActually a downgrade from the previous Air\n,\n\n I find Apple continues to disappoint wit...
7,"\nFaster, Better than last ipad or android tab...",\n\n Nothing beats the speed and functionalit...
8,\nHappy camper!\n,\n\n I was so nervous buying an iPad over Ama...
9,\nGood 💜\n,"\n\n Good iPad; it’s fast, the display looks ..."


**Processing data to lower case and for removing punctuation,numbers & special characters**

---


**Stopwords**, **spelling** **correction**, **stemming** & **Lemmatization**

In [15]:
import re
df['Processing data'] = df['detailed Review'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df
df['Processing data'] = df['Processing data'].str.replace('[^\w\s]','')
df
df['Processing data'] = df['Processing data'].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))
df['Processing data'] = df['Processing data'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x ))
df

,overview of customer review,detailed Review,Processing data
0,\nSmall improvements combine for a great exper...,"\n\n This new iPad Air, on the surface, isn't...",this new ipad air on the surface isnt anything...
1,"\nGreat product, delivery concerning\n","\n\n Having had ipad , ipad air 2, I would ha...",having had ipad ipad air i would have to say...
2,\nnot to user friendly for person over 65 olds...,"\n\n doesn't work that great slow, to many ap...",doesnt work that great slow to many apps perso...
3,"\nGreat for the casuals, and then some\n",\n\n I purchased this as a sort-of alternativ...,i purchased this as a sortof alternative to my...
4,\niPad Air 4 Rocks\n,\n\n I held off replacing my iPad Air 2 in ho...,i held off replacing my ipad air in hopes the...
5,\nThe New iPad Air Is A Hit!!\n,\n\n I think this particular iPad will be a p...,i think this particular ipad will be a pretty ...
6,\nActually a downgrade from the previous Air\n,\n\n I find Apple continues to disappoint wit...,i find apple continues to disappoint with thei...
7,"\nFaster, Better than last ipad or android tab...",\n\n Nothing beats the speed and functionalit...,nothing beats the speed and functionality of t...
8,\nHappy camper!\n,\n\n I was so nervous buying an iPad over Ama...,i was so nervous buying an ipad over amazon bu...
9,\nGood 💜\n,"\n\n Good iPad; it’s fast, the display looks ...",good ipad its fast the display looks really go...


In [17]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
from nltk.corpus import stopwords
rev = stopwords.words('english')
df['Processing data'] = df['Processing data'].apply(lambda x: " ".join(x for x in x.split() if x not in rev))
df

,overview of customer review,detailed Review,Processing data
0,\nSmall improvements combine for a great exper...,"\n\n This new iPad Air, on the surface, isn't...",new ipad air surface isnt anything groundbreak...
1,"\nGreat product, delivery concerning\n","\n\n Having had ipad , ipad air 2, I would ha...",ipad ipad air would say best one yet compared ...
2,\nnot to user friendly for person over 65 olds...,"\n\n doesn't work that great slow, to many ap...",doesnt work great slow many apps person many a...
3,"\nGreat for the casuals, and then some\n",\n\n I purchased this as a sort-of alternativ...,purchased sortof alternative laptop something ...
4,\niPad Air 4 Rocks\n,\n\n I held off replacing my iPad Air 2 in ho...,held replacing ipad air hopes air would worth ...
5,\nThe New iPad Air Is A Hit!!\n,\n\n I think this particular iPad will be a p...,think particular ipad pretty big deal looking ...
6,\nActually a downgrade from the previous Air\n,\n\n I find Apple continues to disappoint wit...,find apple continues disappoint newest product...
7,"\nFaster, Better than last ipad or android tab...",\n\n Nothing beats the speed and functionalit...,nothing beats speed functionality new ipad pro...
8,\nHappy camper!\n,\n\n I was so nervous buying an iPad over Ama...,nervous buying ipad amazon im happy got ipad u...
9,\nGood 💜\n,"\n\n Good iPad; it’s fast, the display looks ...",good ipad fast display looks really good feels...


In [21]:
#spelling
from textblob import TextBlob
df['Processing data'].apply(lambda x: str(TextBlob(x).correct()))


0    new pad air surface isn anything groundbreakin...
1    pad pad air would say best one yet compared pr...
2    doesn work great slow many apes person many ap...
3    purchased sort alternative lawton something su...
4    held replacing pad air hopes air would worth f...
5    think particular pad pretty big deal looking a...
6    find apple continues disappoint newest product...
7    nothing beats speed functionality new pad proc...
8    nervous buying pad amazon in happy got pad usu...
9    good pad fast display looks really good feels ...
Name: Processing data, dtype: object

In [ ]:
#stemming
from nltk.stem import PorterStemmer
s = PorterStemmer()
df['Processing data'].apply(lambda x: " ".join([s.stem(word) for word in x.split()]))

In [22]:
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [23]:
from textblob import Word
df['Processing data'] = df['Processing data'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
df

,overview of customer review,detailed Review,Processing data
0,\nSmall improvements combine for a great exper...,"\n\n This new iPad Air, on the surface, isn't...",new ipad air surface isnt anything groundbreak...
1,"\nGreat product, delivery concerning\n","\n\n Having had ipad , ipad air 2, I would ha...",ipad ipad air would say best one yet compared ...
2,\nnot to user friendly for person over 65 olds...,"\n\n doesn't work that great slow, to many ap...",doesnt work great slow many apps person many a...
3,"\nGreat for the casuals, and then some\n",\n\n I purchased this as a sort-of alternativ...,purchased sortof alternative laptop something ...
4,\niPad Air 4 Rocks\n,\n\n I held off replacing my iPad Air 2 in ho...,held replacing ipad air hope air would worth f...
5,\nThe New iPad Air Is A Hit!!\n,\n\n I think this particular iPad will be a p...,think particular ipad pretty big deal looking ...
6,\nActually a downgrade from the previous Air\n,\n\n I find Apple continues to disappoint wit...,find apple continues disappoint newest product...
7,"\nFaster, Better than last ipad or android tab...",\n\n Nothing beats the speed and functionalit...,nothing beat speed functionality new ipad proc...
8,\nHappy camper!\n,\n\n I was so nervous buying an iPad over Ama...,nervous buying ipad amazon im happy got ipad u...
9,\nGood 💜\n,"\n\n Good iPad; it’s fast, the display looks ...",good ipad fast display look really good feel l...


In [32]:
import nltk
nltk.download('punkt')
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

 **POS-tag features**

In [35]:
from nltk.tokenize import word_tokenize
x = []
for sent in df['Processing data']:
  text = word_tokenize(sent)
  x.append(nltk.pos_tag(text))
x

[[('new', 'JJ'),
  ('ipad', 'NN'),
  ('air', 'NN'),
  ('surface', 'NN'),
  ('isnt', 'NN'),
  ('anything', 'NN'),
  ('groundbreaking', 'VBG'),
  ('bit', 'NN'),
  ('faster', 'JJ'),
  ('bit', 'NN'),
  ('thinner', 'JJ'),
  ('bit', 'RB'),
  ('lighter', 'JJ'),
  ('bit', 'NN'),
  ('shinier', 'JJ'),
  ('tablet', 'NN'),
  ('ive', 'NN'),
  ('used', 'VBN'),
  ('taken', 'VBN'),
  ('altogetherand', 'RB'),
  ('taken', 'VBN'),
  ('along', 'RP'),
  ('secondgen', 'JJ'),
  ('apple', 'NN'),
  ('pencilit', 'NN'),
  ('feel', 'VBP'),
  ('like', 'IN'),
  ('entirely', 'RB'),
  ('new', 'JJ'),
  ('experience', 'NN'),
  ('havent', 'NN'),
  ('struck', 'VBD'),
  ('new', 'JJ'),
  ('device', 'NN'),
  ('whileover', 'IN'),
  ('year', 'NN'),
  ('ive', 'NN'),
  ('owned', 'VBD'),
  ('three', 'CD'),
  ('four', 'CD'),
  ('ipads', 'NNS'),
  ('android', 'JJ'),
  ('tablet', 'NN'),
  ('convertible', 'JJ'),
  ('touchscreen', 'JJ'),
  ('laptop', 'JJ'),
  ('running', 'VBG'),
  ('various', 'JJ'),
  ('os', 'JJ'),
  ('ive', 'JJ'),
 

In [72]:
Adj = []
Adv = []
Cor_con = []
Sub_con = []
In = []
N = []
V = []
Pp = []
pre_det = []
Det = []
for value in pos:
  AdjCount = 0
  AdvCount = 0
  Cor_conCount = 0
  Sub_conCount = 0
  InCount = 0
  NCount = 0
  VCount = 0
  PpCount = 0
  pre_detCount = 0
  DetCount = 0
  for x,y in value:
    if y == 'JJ':
      AdjCount = AdjCount + 1
    elif y == 'RB':
      AdvCount = AdvCount + 1
    elif y == 'CC':
      Cor_conCount = Cor_conCount + 1
    elif y == 'IN':
      Sub_conCount = Sub_conCount + 1
    elif y == 'UH':
      InCount = IntCount + 1
    elif y == 'NN':
      NCount = NCount + 1
    elif y == 'VR':
      VCount = VCount + 1
    elif y == 'PRP':
      PpCount = PpCount + 1
    elif y == 'PDT':
      pre_detCount = pre_detCount + 1
    elif y == 'DT':
      DetCount = DetCount + 1
  Adj.append(AdjCount)
  Adv.append(AdvCount)
  Cor_con.append(Cor_conCount)
  Sub_con.append(Sub_conCount)
  In.append(InCount)
  N.append(NCount)
  V.append(VCount)
  Pp.append(PpCount)
  pre_det.append(pre_detCount)
  Det.append(DetCount)
df['Adjectives'] = Adj
df['Adverbs'] = Adv
df['Cordinating Conjunctions'] = Cor_con
df['Subordinating Conjuctions'] = Sub_con
df['Interjections'] = In
df['Nouns'] = N
df['Verbs'] = V
df['Personal Pronouns'] = Pp
df['Predeterminers'] = pre_det
df['Determiners'] = Det
df

  


,overview of customer review,detailed Review,Processing data,Adjectives,Adverbs,Cordinating Conjunctions,Subordinating Conjuctions,Interjections,Nouns,Verbs,Personal Pronouns,Predeterminers,Determiners,Number of Right Branching Nodes
0,\nSmall improvements combine for a great exper...,"\n\n This new iPad Air, on the surface, isn't...",new ipad air surface isnt anything groundbreak...,72,37,1,10,0,114,0,0,0,0,0
1,"\nGreat product, delivery concerning\n","\n\n Having had ipad , ipad air 2, I would ha...",ipad ipad air would say best one yet compared ...,20,7,1,2,0,31,0,0,0,0,0
2,\nnot to user friendly for person over 65 olds...,"\n\n doesn't work that great slow, to many ap...",doesnt work great slow many apps person many a...,11,5,0,1,0,20,0,0,0,0,0
3,"\nGreat for the casuals, and then some\n",\n\n I purchased this as a sort-of alternativ...,purchased sortof alternative laptop something ...,18,6,0,1,0,20,0,0,0,0,3
4,\niPad Air 4 Rocks\n,\n\n I held off replacing my iPad Air 2 in ho...,held replacing ipad air hope air would worth f...,18,4,1,1,0,28,0,0,0,0,2
5,\nThe New iPad Air Is A Hit!!\n,\n\n I think this particular iPad will be a p...,think particular ipad pretty big deal looking ...,11,5,0,0,0,14,0,0,0,0,1
6,\nActually a downgrade from the previous Air\n,\n\n I find Apple continues to disappoint wit...,find apple continues disappoint newest product...,45,32,0,2,0,92,0,1,0,0,1
7,"\nFaster, Better than last ipad or android tab...",\n\n Nothing beats the speed and functionalit...,nothing beat speed functionality new ipad proc...,5,2,0,0,0,11,0,0,0,0,0
8,\nHappy camper!\n,\n\n I was so nervous buying an iPad over Ama...,nervous buying ipad amazon im happy got ipad u...,9,5,0,1,0,14,0,0,0,0,0
9,\nGood 💜\n,"\n\n Good iPad; it’s fast, the display looks ...",good ipad fast display look really good feel l...,10,3,0,1,0,13,0,0,0,0,0


**Lingustic** **Features** 


> **Right & Left branching nodes,Premodifiers & Postmodifiers,Branching index &Branching Weight,Modification & Modification Weight(constituent types & NPs)**


>**Cordination balance, Density of(pronouns, prepositions,punctuations,auxiliary,conjuctions)**


> **Sentence Length**








In [61]:
pip install spacy


In [63]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package brown to /root/nltk_data...
       |   Unzipping corpora/brown.zip.
       | Downloading package brown_tei to /root/nltk_data...
       |   Unzipping corpora/brown_tei.zip.
       | Downloading package cess_cat to /root/nltk_data...
       |   Unzipping corpora/cess_cat.zip.
       | Downloading package cess_esp to /root/nltk_data...
 

True

In [80]:
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()
Rn = []
nlp = spacy.load("en_core_web_sm")
for sent in df['Processing data']:
  doc = nlp(sent)
  try:
    Rn.append(doc[0].n_rights)
  except:
    Rn.append('No')
df['Right Branching Nodes'] = Rn
df
df['Sentenece Length'] = df['detailed Review'].apply(lambda x: len(x))
df

,overview of customer review,detailed Review,Processing data,Adjectives,Adverbs,Cordinating Conjunctions,Subordinating Conjuctions,Interjections,Nouns,Verbs,Personal Pronouns,Predeterminers,Determiners,Number of Right Branching Nodes,Sentenece Length,Right Branching Nodes
0,\nSmall improvements combine for a great exper...,"\n\n This new iPad Air, on the surface, isn't...",new ipad air surface isnt anything groundbreak...,72,37,1,10,0,114,0,0,0,0,0,3491,0
1,"\nGreat product, delivery concerning\n","\n\n Having had ipad , ipad air 2, I would ha...",ipad ipad air would say best one yet compared ...,20,7,1,2,0,31,0,0,0,0,0,925,0
2,\nnot to user friendly for person over 65 olds...,"\n\n doesn't work that great slow, to many ap...",doesnt work great slow many apps person many a...,11,5,0,1,0,20,0,0,0,0,0,475,0
3,"\nGreat for the casuals, and then some\n",\n\n I purchased this as a sort-of alternativ...,purchased sortof alternative laptop something ...,18,6,0,1,0,20,0,0,0,0,3,625,3
4,\niPad Air 4 Rocks\n,\n\n I held off replacing my iPad Air 2 in ho...,held replacing ipad air hope air would worth f...,18,4,1,1,0,28,0,0,0,0,2,744,2
5,\nThe New iPad Air Is A Hit!!\n,\n\n I think this particular iPad will be a p...,think particular ipad pretty big deal looking ...,11,5,0,0,0,14,0,0,0,0,1,308,1
6,\nActually a downgrade from the previous Air\n,\n\n I find Apple continues to disappoint wit...,find apple continues disappoint newest product...,45,32,0,2,0,92,0,1,0,0,1,2332,1
7,"\nFaster, Better than last ipad or android tab...",\n\n Nothing beats the speed and functionalit...,nothing beat speed functionality new ipad proc...,5,2,0,0,0,11,0,0,0,0,0,279,0
8,\nHappy camper!\n,\n\n I was so nervous buying an iPad over Ama...,nervous buying ipad amazon im happy got ipad u...,9,5,0,1,0,14,0,0,0,0,0,436,0
9,\nGood 💜\n,"\n\n Good iPad; it’s fast, the display looks ...",good ipad fast display look really good feel l...,10,3,0,1,0,13,0,0,0,0,0,317,0
